In [6]:
!issparse(sparse([1],[1],[0]))

false

In [10]:
true == true || false == true

true

In [12]:
?|

search: 

```
|(x, y)
```

Bitwise or


| |>



In [17]:
?lu

search: 

```
lu(A) -> L, U, p
```

Compute the LU factorization of `A`, such that `A[p,:] = L*U`.


lu lufact lufact! flush flush_cstdio ClusterManager values include



In [20]:
A = speye(3)

3x3 sparse matrix with 3 Float64 entries:
	[1, 1]  =  1.0
	[2, 2]  =  1.0
	[3, 3]  =  1.0

In [22]:
lufact(A,pivot=Val{false})

LoadError: LoadError: ArgumentError: function lufact does not accept keyword arguments
while loading In[22], in expression starting on line 1

In [38]:
A = [1;2;3]

3-element Array{Int64,1}:
 1
 2
 3

In [35]:
splice!(A,2:3)

2-element Array{Int64,1}:
 2
 3

In [42]:
n = 10
p = 3
sparse(n-p,p)

LoadError: LoadError: MethodError: `sparse` has no method matching sparse(::Int64, ::Int64)
Closest candidates are:
  sparse(::Any, ::Any, !Matched::Number)
  sparse(::Any, ::Any, !Matched::AbstractArray{Bool,1}, !Matched::Any, !Matched::Any)
  sparse(::Any, ::Any, !Matched::AbstractArray{T,1})
  ...
while loading In[42], in expression starting on line 3

In [77]:
?lufact

search: 

```rst
..  lufact(A [,pivot=Val{true}]) -> F

Compute the LU factorization of ``A``. The return type of ``F`` depends on the type of ``A``. In most cases, if ``A`` is a subtype ``S`` of AbstractMatrix with an element type ``T`` supporting ``+``, ``-``, ``*`` and ``/`` the return type is ``LU{T,S{T}}``. If pivoting is chosen (default) the element type should also support ``abs`` and ``<``. When ``A`` is sparse and have element of type ``Float32``, ``Float64``, ``Complex{Float32}``, or ``Complex{Float64}`` the return type is ``UmfpackLU``. Some examples are shown in the table below.

======================= ========================= ========================================
Type of input ``A``     Type of output ``F``      Relationship between ``F`` and ``A``
======================= ========================= ========================================
:func:`Matrix`           ``LU``                   ``F[:L]*F[:U] == A[F[:p], :]``
:func:`Tridiagonal`      ``LU{T,Tridiagonal{T}}`` ``F[:L]*F[:U] == A[F[:p], :]``
:func:`SparseMatrixCSC`  ``UmfpackLU``            ``F[:L]*F[:U] == (F[:Rs] .* A)[F[:p], F[:q]]``
======================= ========================= ========================================

The individual components of the factorization ``F`` can be accessed by indexing:

=========== ======================================= ====== ======================== =============
Component   Description                             ``LU`` ``LU{T,Tridiagonal{T}}`` ``UmfpackLU``
=========== ======================================= ====== ======================== =============
``F[:L]``   ``L`` (lower triangular) part of ``LU``    ✓            ✓                        ✓
``F[:U]``   ``U`` (upper triangular) part of ``LU``    ✓            ✓                        ✓
``F[:p]``   (right) permutation ``Vector``             ✓            ✓                        ✓
``F[:P]``   (right) permutation ``Matrix``             ✓            ✓
``F[:q]``   left permutation ``Vector``                                                      ✓
``F[:Rs]``  ``Vector`` of scaling factors                                                    ✓
``F[:(:)]`` ``(L,U,p,q,Rs)`` components                                                      ✓
=========== ======================================= ====== ======================== =============

================== ====== ======================== =============
Supported function ``LU`` ``LU{T,Tridiagonal{T}}`` ``UmfpackLU``
================== ====== ======================== =============
     ``/``            ✓
     ``\``            ✓                       ✓             ✓
     ``cond``         ✓                                     ✓
     ``det``          ✓                       ✓             ✓
     ``logdet``       ✓                       ✓
     ``logabsdet``    ✓                       ✓
     ``size``         ✓                       ✓
================== ====== ======================== =============
```


lufact lufact!



In [73]:
"""
    [L,U,Q] = luq(A,tol)
Calculate the following decomposition
A = L |Ubar  0 | Q
      |0     0 |
where Ubar is a square invertible matrix
and matrices L, Q are invertible.
Based on lu decomposition.

Inputs:
* `A`: a sparse matrix
* `tol`: used to separate zero and nonzero values

Note: Julia's lufact always does column pivoting
for sparse matrices.

Ported from Pawel Kowal:
Copyright  (c) Pawel Kowal (2006)
All rights reserved
LREM_SOLVE toolbox is available free for noncommercial academic use only.
pkowal3@sgh.waw.pl
"""
function luq(A,tol)
    A = sparse(A)
    n,m = size(A)

    ###################
    ## Special cases ##
    ###################
    if n == 0 || m == 0
        L = speye(n)
        U = A
        Q = speye(m)
        return L,U,Q
    end

    ######################
    ## LU decomposition ##
    ######################
    L,U,P,Q,Rs = lufact(A)[:(:)]
    Q = Q'

    p = n - size(L,2)
    LL = [spzeros(n-p,p);speye(p)]
    L = [P'*L P[n-p+1:n,:]']
    U = [U;spzeros(p,m)]

    ##########################
    ## Find zero pivot rows ##
    ##########################
    if size(U,1) == 1 || size(U,2) == 1
        S = U[1,1]
    else
        S = diag(U)
    end
    I = find(abs(S).>tol)
    Jl = setdiff(1:n,I)
    Jq = setdiff(1:m,I)

    Ubar1 = U[I,I]
    Ubar2 = U[Jl,Jq]
    Qbar1 = Q[I,:]
    Lbar1 = L[:,I]

    ##########################################
    ## Eliminate nonzero elements below and ##
    ## on right of invertible block of U,   ##
    ## and update L and Q                   ##
    ##########################################
    if ~isempty(I)
        Utmp = U[I,Jq]
        X = Ubar1'\U[Jl,I]'
        Ubar2 = Ubar2 - X'*Utmp
        Lbar1 = Lbar1 + L[:,Jl]*X'

        X = Ubar\Utmp
        Qbar1 = Qbar1 + X*Q[Jq,:]
        Utmp = []
        X = []
    end

    ################################################
    ## Find rows and cols with only zero elements ##
    ################################################
    I2 = find(maxabs(Ubar2,2).>tol);
    I5 = find(maxabs(Ubar2,1).>tol);

    I3 = Jl[I2]
    I4 = Jq[I5]
    Jq[I5] = []
    Jl[I2] = []
    U = []

    ########################################
    ## Find part of U which is not in the ##
    ## required form                      ##
    ########################################

    A = Ubar2[I2,I5]

    ####################################
    ## Perform luq decomposition on A ##
    ####################################
    L1,U1,Q1 = luq(A,pivot,tol)

    ##################
    ## Update L,U,Q ##
    ##################
    Lbar2 = L[:,I3]*L1
    Qbar2 = Q1*Q[I4,:]
    L = [Lbar1 Lbar2 L[:,Jl]]
    Q = [Qbar1; Qbar2; Q[Jq,:]]

    n1,n2,m2 = length(I), length(I3), length(I4)
    U = [Ubar1 spzeros(n1,m-n1);spzeros(n2,n1) U1 spzeros(n2,m-n1-m2);spzers(n-n1-n2,m)]

    return L,U,Q
end


luq (generic function with 1 method)

In [74]:
A = sprand(10,20,0.4)
L,U,Q = luq(A',1e-10)

LoadError: LoadError: BoundsError: attempt to access 1x10 Array{Int64,2}:
 3  1  6  8  10  4  9  2  5  7
  at index [[1,2,3,4,5,6,7,8,9,10],Colon()]
while loading In[74], in expression starting on line 2

In [76]:
lufact(A')

UMFPACK LU Factorization of a 20-by-10 sparse matrix
Ptr{Void} @0x000000000456c240


In [66]:
L,U,p = lu(full(A))

(
10x10 Array{Float64,2}:
 1.0         0.0       0.0       …   0.0         0.0       0.0      0.0
 0.0         1.0       0.0           0.0         0.0       0.0      0.0
 0.641221    0.0       1.0           0.0         0.0       0.0      0.0
 0.00103891  0.0       0.0           0.0         0.0       0.0      0.0
 0.0         0.0       0.0           0.0         0.0       0.0      0.0
 0.227658    0.0       0.930291  …   0.0         0.0       0.0      0.0
 0.333305    0.0       0.475885      1.0         0.0       0.0      0.0
 0.0866356   0.0       0.943914     -0.140653    1.0       0.0      0.0
 0.895525    0.231839  0.0          -0.918297   -0.493573  1.0      0.0
 0.149049    0.0       0.0           0.0746773   0.531549  0.11283  1.0,

10x20 Array{Float64,2}:
 0.743768  0.0       0.0        0.681562   …   0.010239     0.372481   
 0.0       0.849332  0.0        0.525406       0.0          0.0        
 0.0       0.0       0.978995  -0.0416025     -0.00656549  -0.238843   
 0.0       0

In [68]:
maxabs(L*U - full(A[p,:]))

1.1102230246251565e-16